<a href="https://colab.research.google.com/github/superguine/Object_Detection_Project/blob/main/Train_3.3.1.2.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object Detection from Images Using TensorFlow: A Deep Learning Approach

#### Final Year Project Notebook  
**B.Tech – Department of Computer Science & Engineering (CSE) & Information Technology (IT)**  
**Bankura Unnayani Institute of Engineering (BUIE)**  

---

### Project Members
- Shawon Roy  
- Ishika Lohar  
- Shreya Dey  
- Tiasa Das  

### Project Supervisor  
Mr. Amit Goswami  
Assistant Professor  

### Notebook Prepared By  
Shawon Roy

---

> **Disclaimer:**  
> _While the overall structure and most of the code in this notebook have been developed by me (Shawon Roy), a few sections (specifically 2–4 key code cells) have been adapted or reused from open-source projects and contributions by the TensorFlow developer community and other ML developers._  
>
> _Given the complexity of setting up a complete object detection pipeline, these integrations were essential to meet the project goals efficiently and accurately._  
> _Full credit and acknowledgment go to the respective developers and original sources._

**Last Updated:** May 24, 2025  
**Changelog:** [View on GitHub](https://github.com/superguine/ODProject/blob/main/Changelog.md)


# Installation Section

In [ ]:
# Checks for GPU and lists them.
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
# Clone the tensorflow models repository from GitHub
!git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Copy setup files into models/research folder.
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .

In [ ]:
# Modify setup.py file to install the tf-models-official repository targeted at TF v2.15.0
import re
with open('/content/models/research/object_detection/packages/tf2/setup.py') as f:
    s = f.read()

with open('/content/models/research/setup.py', 'w') as f:
    # Set fine_tune_checkpoint path
    s = re.sub('tf-models-official>=2.5.1',
               'tf-models-official==2.15.0', s)
    f.write(s)

⚠️**NOTE :**  
Below cell will ask for a session restart. Make sure that the cell completes execution before you restart the session.

In [ ]:
# Need to do a temporary fix with PyYAML because Colab isn't able to install PyYAML v5.4.1
!pip install pyyaml==5.3.1
!pip install /content/models/research/

Below cell will take 10-15 minutes to complete execution.

In [ ]:
# Uninstall existing TensorFlow
!pip uninstall -y tensorflow tensorflow-estimator tensorboard

# Install NVIDIA CUDA Toolkit 11.8 (needed for TensorFlow 2.15.0)
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin
!sudo mv cuda-ubuntu2204.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda-repo-ubuntu2204-11-8-local_11.8.0-520.61.05-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu2204-11-8-local_11.8.0-520.61.05-1_amd64.deb
!sudo cp /var/cuda-repo-ubuntu2204-11-8-local/cuda-*-keyring.gpg /usr/share/keyrings/
!sudo apt-get update
!sudo apt-get install -y cuda-toolkit-11-8

# Install cuDNN 8.6 (needed for TensorFlow 2.15.0)
!wget https://developer.download.nvidia.com/compute/redist/cudnn/v8.6.0/local_installers/cudnn-linux-x86_64-8.6.0.163_cuda11-archive.tar.xz
!tar -xvf cudnn-linux-x86_64-8.6.0.163_cuda11-archive.tar.xz
!sudo cp -r cudnn-linux-x86_64-8.6.0.163_cuda11-archive/include/* /usr/local/cuda/include/
!sudo cp -r cudnn-linux-x86_64-8.6.0.163_cuda11-archive/lib/* /usr/local/cuda/lib64/
!sudo chmod a+r /usr/local/cuda/include/cudnn*.h /usr/local/cuda/lib64/libcudnn*

# Set Environment Variables
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda-11.8'
os.environ['PATH'] += ':/usr/local/cuda-11.8/bin'
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-11.8/lib64:' + os.environ.get('LD_LIBRARY_PATH', '')

# Install TensorFlow 2.15.0 (GPU Version)
!pip install tensorflow[and-cuda]==2.15.0 --extra-index-url https://pypi.nvidia.com

# Verify GPU Availability
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))


In [ ]:
# Optional - just to see if everything is good till now.
!nvidia-smi
!nvcc --version

⚠️NOTE :
Below cell will ask for a session restart. Make sure that the cell completes execution before you restart the session. If it doesn't, asks automatically, you have to manually click the [**RESTART SESSION**] button appears in the output section.

In [ ]:
#Uninstall & reinstall compatible protobuf version.
!pip uninstall -y protobuf
!pip install protobuf==3.20.3

# Set environment type.
%env PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python

#### Test installation

In [ ]:
# This script tests all of the installations.
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

In [ ]:
# Optional - just for checking the TF version & GPU.
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPUs available:", len(tf.config.list_physical_devices('GPU')))

# Dataset collecting Section

**Important‼️**  
Upload your dataset (images.zip file) before running next cell.
  The dataset orientation should be like :
```
images.zip
    |
  images
    |----- Img01.jpg
    |----- Img01.xml
    |----- Img02.jpg
    |----- Img02.xml
    :
    :.. .

```

In [ ]:
# Make directories for different datasets.
!mkdir /content/images
!unzip -q images.zip -d /content/images/all
!mkdir /content/images/train; mkdir /content/images/validation; mkdir /content/images/test

In [ ]:
# Download & run the data splitting script
!wget https://raw.githubusercontent.com/superguine/ODProject/main/split.py
!python split.py

In [ ]:
# Download data conversion scripts
! wget https://raw.githubusercontent.com/superguine/ODProject/main/csv_convert.py
! wget https://raw.githubusercontent.com/superguine/ODProject/main/tfrecord.py

Next cell creates a  "labelmap.txt" file with a list of classes the object detection model will detect.You have to write each class in a new line (between '**cat <<EOF >> /content /labelmap.txt** '  & ' **EOF** ' )which are present in the dataset.  
> Suppose your classes are: ***Dry, EatenByInsects, Fresh, PartiallyDry, Wrinkled***.
then the cell should look like:  
```
%%bash
cat <<EOF >> /content/labelmap.txt
Dry
EatenByInsects
Fresh
PartiallyDry
Wrinkled
EOF
```

In [ ]:
# Create "labelmap.txt"
%%bash
cat <<EOF >> /content/labelmap.txt
Dry
EatenByInsects
Fresh
PartiallyDry
Wrinkled
EOF

In [ ]:
# Create CSV data files and TFRecord files
!python3 csv_convert.py
!python3 tfrecord.py --csv_input=images/train_labels.csv --labelmap=labelmap.txt --image_dir=images/train --output_path=train.tfrecord
!python3 tfrecord.py --csv_input=images/validation_labels.csv --labelmap=labelmap.txt --image_dir=images/validation --output_path=val.tfrecord

In [ ]:
# Store the locations of the TFRecord and labelmap files as variables
train_record_fname = '/content/train.tfrecord'
val_record_fname = '/content/val.tfrecord'
label_map_pbtxt_fname = '/content/labelmap.pbtxt'

# Training Section

#### Configuring for training

> 🔵 **Note (April 2025):**  
> The `efficientdet-d0` model currently cannot be trained due to conflicts in its configuration file.  
> ⚠️ It's recommended to **avoid selecting this specific model** until the issue is resolved.

In [ ]:
# Change the chosen_model variable to deploy different models available in the TF2 object detection zoo
chosen_model = 'ssd-mobilenet-v2'

MODELS_CONFIG = {
    'ssd-mobilenet-v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
    },
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
    },
    'ssd-mobilenet-v2-fpnlite-320': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
    },
}

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

In nex cell we'll download the pretrained model file and configuration file.

In [ ]:
# Create "mymodel" folder for holding pre-trained weights and configuration files
%mkdir /content/models/mymodel/
%cd /content/models/mymodel/

# Download pre-trained model weights
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

# Download training configuration file for model
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

In [ ]:
# Set training parameters for the model
num_steps = 50000

if chosen_model == 'efficientdet-d0':
  batch_size = 4
else:
  batch_size = 16

In [ ]:
# Set file locations and get number of classes for config file
pipeline_fname = '/content/models/mymodel/' + base_pipeline_file
fine_tune_checkpoint = '/content/models/mymodel/' + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    import tensorflow as tf  # Import TensorFlow explicitly

    with tf.io.gfile.GFile(pbtxt_fname, 'r') as fid:  # Use tf.io.gfile.GFile
        label_map_string = fid.read()

    from object_detection.protos import string_int_label_map_pb2
    from google.protobuf import text_format

    label_map = string_int_label_map_pb2.StringIntLabelMap()
    text_format.Merge(label_map_string, label_map)

    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True
    )
    category_index = label_map_util.create_category_index(categories)

    return len(category_index.keys())

# Call the function
num_classes = get_num_classes(label_map_pbtxt_fname)
print('Total classes:', num_classes)


In [ ]:
# Create custom configuration file by writing the dataset, model checkpoint, and training parameters into the base pipeline file
import re

%cd /content/models/mymodel
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:

    # Set fine_tune_checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # Set tfrecord files for train and test datasets
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)

    # Set label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set batch_size
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)

    # Set number of classes num_classes
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    # Change fine-tune checkpoint type from "classification" to "detection"
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)

    # If using ssd-mobilenet-v2, reduce learning rate (because it's too high in the default config file)
    if chosen_model == 'ssd-mobilenet-v2':
      s = re.sub('learning_rate_base: .8',
                 'learning_rate_base: .08', s)

      s = re.sub('warmup_learning_rate: 0.13333',
                 'warmup_learning_rate: .026666', s)

    # If using efficientdet-d0, use fixed_shape_resizer instead of keep_aspect_ratio_resizer (because it isn't supported by TFLite)
    if chosen_model == 'efficientdet-d0':
      #s = re.sub('keep_aspect_ratio_resizer', 'fixed_shape_resizer', s)
      s = re.sub('pad_to_max_dimension: true', '', s)
      s = re.sub('min_dimension', 'height', s)
      s = re.sub('max_dimension', 'width', s)

    f.write(s)


Optional - at this point, you can see the custom configuration file by just clicking this: ` /content/models/mymodel/pipeline_file.config  `.

In [ ]:
# Set the path to the custom config file and the directory to store training checkpoints in
pipeline_file = '/content/models/mymodel/pipeline_file.config'
model_dir = '/content/training/'

In [ ]:
# Modify conflicting files.
import os

# Define the file path
file_path = "/usr/local/lib/python3.11/dist-packages/tf_slim/data/tfexample_decoder.py"

# Check if the file exists
if not os.path.exists(file_path):
    print(f"Error: {file_path} not found.")
else:
    # Read the file contents
    with open(file_path, "r") as f:
        content = f.read()

    # Apply fixes
    content = content.replace(
        "from tensorflow.python.ops import control_flow_ops", "import tensorflow as tf"
    )
    content = content.replace("control_flow_ops.case", "tf.case")
    content = content.replace(
        "tf.case(pred_fn_pairs, default=check_jpeg, exclusive=True)",
        "tf.switch_case(branch_index=tf.constant(0), branch_fns={0: lambda: check_jpeg()})"
    )
    content = content.replace("control_flow_ops.cond", "tf.cond")

    # Write the modified content back to the file
    with open(file_path, "w") as f:
        f.write(content)

    print(" Modifications applied successfully.")


In the next cell, we'll launch [TensorBoard](https://www.tensorflow.org/tensorboard) to monitor training progress.

> 🔵 **Note:**  
> TensorBoard **won’t display anything yet**, since training hasn’t started.  
> Once training begins, you'll start seeing logs like the example below in the output:
```
INFO:tensorflow:Step 22000 per-step time 0.151s
I0412 07:38:16.052670 15402206208000 model_lib_v2.py:705] Step 22000 per-step time 0.151s
INFO:tensorflow:{'Loss/classification_loss': 0.2932836,
 'Loss/localization_loss': 0.17571348,
 'Loss/regularization_loss': 0.21325982,
 'Loss/total_loss': 0.68225694,
 'learning_rate': 0.05035276}
I0412 07:38:16.052977 15402206208000 model_lib_v2.py:708] {'Loss/classification_loss': 0.2932836,
 'Loss/localization_loss': 0.17571348,
 'Loss/regularization_loss': 0.21325982,
 'Loss/total_loss': 0.68225694,
 'learning_rate': 0.05035276}
 ```
  Then come back and click the refresh button to see the model's overall loss.

In [ ]:
#launching Tensorboard session
%load_ext tensorboard
%tensorboard --logdir '/content/training/train'

#### Start training

In [ ]:
# Run training!
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1

# Model exportation & testing Section

#### Export model

In [ ]:
# Make a directory to store the trained TensorFlow model
!mkdir /content/custom_model
output_directory = '/content/custom_model'

# Path to training directory (the conversion script automatically chooses the highest checkpoint file)
last_model_path = '/content/training'

!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {last_model_path} \
    --pipeline_config_path {pipeline_file} \
    --output_directory {output_directory}

print("\n Done exporting ")

####Inference test images

In [ ]:
# Import packages
import os
import cv2
import numpy as np
import sys
import glob
import random
import tensorflow as tf

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

### Define function for inferencing with TensorFlow model and displaying results

def tf_detect_images(modelpath, imgpath, lblpath, min_conf=0.5, num_test_images=10, savepath='/content/results', txt_only=False):

    # Grab filenames of all images in test folder
    images = glob.glob(imgpath + '/*.jpg') + glob.glob(imgpath + '/*.JPG') + glob.glob(imgpath + '/*.png') + glob.glob(imgpath + '/*.bmp')

    # Load the label map into memory
    with open(lblpath, 'r') as f:
        labels = [line.strip() for line in f.readlines()]

    # Load the TensorFlow model
    model = tf.saved_model.load(modelpath)
    infer = model.signatures['serving_default']

    # Randomly select test images
    images_to_test = random.sample(images, num_test_images)

    # Loop over every image and perform detection
    for image_path in images_to_test:

        # Load image and resize to expected shape [1xHxWx3]
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        imH, imW, _ = image.shape
        input_tensor = tf.convert_to_tensor(image_rgb)
        input_tensor = input_tensor[tf.newaxis, ...]

        # Perform the actual detection by running the model with the image as input
        detections = infer(input_tensor)

        # Retrieve detection results
        boxes = detections['detection_boxes'][0].numpy() # Bounding box coordinates of detected objects
        classes = detections['detection_classes'][0].numpy().astype(np.int32) # Class index of detected objects
        scores = detections['detection_scores'][0].numpy() # Confidence of detected objects

        detection_results = []

        # Loop over all detections and draw detection box if confidence is above minimum threshold
        for i in range(len(scores)):
            if ((scores[i] > min_conf) and (scores[i] <= 1.0)):

                # Get bounding box coordinates and draw box
                # Coordinates are normalized, so we multiply them by the image dimensions to get pixel values
                ymin = int(max(1, (boxes[i][0] * imH)))
                xmin = int(max(1, (boxes[i][1] * imW)))
                ymax = int(min(imH, (boxes[i][2] * imH)))
                xmax = int(min(imW, (boxes[i][3] * imW)))

                cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (10, 255, 0), 2)

                # Draw label
                object_name = labels[classes[i] - 1] # Adjust for zero-indexing in labels
                label = '%s: %d%%' % (object_name, int(scores[i] * 100)) # Example: 'person: 72%'
                labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2) # Get font size
                label_ymin = max(ymin, labelSize[1] + 10) # Make sure not to draw label too close to top of window
                cv2.rectangle(image, (xmin, label_ymin - labelSize[1] - 10),
                              (xmin + labelSize[0], label_ymin + baseLine - 10),
                              (255, 255, 255), cv2.FILLED) # Draw white box to put label text in
                cv2.putText(image, label, (xmin, label_ymin - 7),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2) # Draw label text

                detection_results.append([object_name, scores[i], xmin, ymin, xmax, ymax])

        # Display or save the image with detections
        if not txt_only: # "text_only" controls whether we want to display the image results or just save them in .txt files
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(12, 16))
            plt.imshow(image)
            plt.show()

        # Save detection results in .txt files (for calculating mAP)
        else:
            # Get filenames and paths
            image_fn = os.path.basename(image_path)
            base_fn, ext = os.path.splitext(image_fn)
            txt_result_fn = base_fn + '.txt'
            txt_savepath = os.path.join(savepath, txt_result_fn)

            # Write results to text file
            # (Using format defined by https://github.com/Cartucho/mAP, which will make it easy to calculate mAP)
            with open(txt_savepath, 'w') as f:
                for detection in detection_results:
                    f.write('%s %.4f %d %d %d %d\n' %
                            (detection[0], detection[1], detection[2],
                             detection[3], detection[4], detection[5]))

    return


In [ ]:
# Set up variables for running user's model
PATH_TO_IMAGES = '/content/images/test'   # Path to test images folder
PATH_TO_MODEL = '/content/custom_model/saved_model'   # Path to the SavedModel directory
PATH_TO_LABELS = '/content/labelmap.txt'   # Path to labelmap.txt file
min_conf_threshold = 0.5   # Confidence threshold (try changing this to 0.01 if you don't see any detection results)
images_to_test = 10   # Number of images to run detection on

# Run inferencing function!
tf_detect_images(PATH_TO_MODEL, PATH_TO_IMAGES, PATH_TO_LABELS, min_conf_threshold, images_to_test)


#### Calculate mAP

Now we have a visual sense of how our model performs on test images.Now wer'e going to calculate the accuracy of our model through a popular method, which is "mean average precision" (mAP). We'll use the mAP calculator tool at [here](https://github.com/Cartucho/mAP) to determine our model's mAP score.

First, we need to clone the repository and remove its existing example data. We'll also download a script I wrote for interfacing with the calculator.

In [ ]:
%%bash
git clone https://github.com/Cartucho/mAP /content/mAP
cd /content/mAP
rm input/detection-results/*
rm input/ground-truth/*
rm input/images-optional/*
wget https://raw.githubusercontent.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/master/util_scripts/calculate_map_cartucho.py

Next, we'll copy the images and annotation data from the **test** folder to the appropriate folders inside the cloned repository. These will be used as the "ground truth data" that our model's detection results will be compared to.


In [ ]:
!cp /content/images/test/* /content/mAP/input/images-optional # Copy images and xml files
!mv /content/mAP/input/images-optional/*.xml /content/mAP/input/ground-truth/  # Move xml files to the appropriate folder

The calculator tool expects annotation data in a format that's different from the Pascal VOC .xml file format we're using. Fortunately, it provides an easy script, `convert_gt_xml.py`, for converting to the expected .txt format.



In [ ]:
# Convert necessary files from .xml to .txt
!python /content/mAP/scripts/extra/convert_gt_xml.py

In [ ]:
# Set up variables for running inference, this time to get detection results saved as .txt files
PATH_TO_IMAGES='/content/images/test'   # Path to test images folder
PATH_TO_MODEL='/content/custom_model/saved_model'   # Path to .tflite model file
PATH_TO_LABELS='/content/labelmap.txt'   # Path to labelmap.txt file
PATH_TO_RESULTS='/content/mAP/input/detection-results' # Folder to save detection results in
min_conf_threshold=0.1   # Confidence threshold

# Use all the images in the test folder
image_list = glob.glob(PATH_TO_IMAGES + '/*.jpg') + glob.glob(PATH_TO_IMAGES + '/*.JPG') + glob.glob(PATH_TO_IMAGES + '/*.png') + glob.glob(PATH_TO_IMAGES + '/*.bmp')
images_to_test = min(500, len(image_list)) # If there are more than 500 images in the folder, just use 500

# Tell function to just save results and not display images
txt_only = True

# Run inferencing function!
print('Starting inference on %d images...' % images_to_test)
tf_detect_images(PATH_TO_MODEL, PATH_TO_IMAGES, PATH_TO_LABELS, min_conf_threshold, images_to_test, PATH_TO_RESULTS, txt_only)
print('Finished inferencing!')

In [ ]:
# Calculate the mAP & store the score in 'mAP_output.txt' file
%cd /content/mAP
!python calculate_map_cartucho.py --labels=/content/labelmap.txt > /content/mAP_output.txt

# Show the scores in cell output.
!cat /content/mAP_output.txt

# Model Downloading Section

#### Download the model

Alright! Now we’ll gather all the necessary components of our trained model and then compress them into a `.zip` file for easy storage or sharing.

In [ ]:
# Move labelmap and pipeline config files into TF model folder and zip it up
!cp /content/labelmap.txt /content/custom_model
!cp /content/labelmap.pbtxt /content/custom_model
!cp /content/models/mymodel/pipeline_file.config /content/custom_model
!cp /content/mAP_output.txt /content/custom_model

# Compress all.
%cd /content
!zip -r custom_model.zip custom_model

Our model is now ready for download and deployment.

In [ ]:
# Dounload the model

from google.colab import files
files.download('/content/custom_model.zip')

**Optional:** You can download the test and validation datasets used during evaluation.  
_It's always a good idea to archive these datasets for future improvement and tracking._

**⚠️Important :**  To download both files google will ask for downloading multiple files. Please click **Allow**

In [ ]:
# Download test & validation dataset.
import shutil
from google.colab import files

# Zip the test folder
shutil.make_archive('/content/test_images', 'zip', '/content/images/test')
# Download the zip
files.download('/content/test_images.zip')

# Zip the validation folder
shutil.make_archive('/content/validation_images', 'zip', '/content/images/validation')
# Download the zip
files.download('/content/validation_images.zip')